# Kapitel 3 - Die "klassischen" Textklassifizierungsverfahren

Wir unterteilen die Klassifizierungsmethoden in <b>probabilistische</b> und <b>nicht-probabilistische</b> Klassifizierungsverfahren.

## 3.? Grundbausteine eines Klassifizierungsverfahren bei sci-kit learn

## 3.? Naive Bayes

### Das Verfahren

Das <b>Naive Bayes</b> Verfahren ist ein Klassifikationsverfahren, welches auf dem <b>Bayesschen Theorem</b> (auch <b>"Satz von Bayes"</b> genannt) von Thomas Bayes basiert. Beim Naive Bayes Verfahren wird <u>naiv</u> (deshalb auch der Name) angenommen, dass ein jedes Attribut nur vom Klassenattribut abhängt. Obwohl das in der Realität selten der Fall ist, liefert Naive Bayes in der Praxis meist gute Werte.<br>
Das Naive Bayes Verfahren besteht nicht nur aus einem einzigen Algorithmus, sondern vielmehr aus einer Familie von Algorithmen, die alle auf dem gleichen Prinzip beruhen. Alle Algorithmen basieren jedoch auf der probabilistischen Methode. Wie wir im vorherigen Kapitel gesehen haben, bedeutet dies, dass Naive Bayes beispielsweise die Zugehörigkeit einer Kategorie zu einem Text für jede Kategorie berechnet und die höchste Wahrscheinlichkeit als Textkategorie zurückgibt. Die Wahrscheinlichkeiten werden durch das <b>Bayessche Theorem</b> ermittelt. Die Formel für das Theorem lautet: <br>

$ P(A|B) = \frac{P(B|A) \ \cdot \ P(A)}{P(B)} $ &nbsp; [<sup>1</sup>](#fn1) <br>



<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; Zwei anschauliche Rechenbeispiele, welche die Formel vertiefen, findet man auf der Wikipedia Seite vom Satz von Bayes: https://de.wikipedia.org/wiki/Satz_von_Bayes.</span>

ICH: binär und multipel!!

### Ein einfaches Beispiel

Um uns mit dem Naive Bayes Verfahren vertraut zu machen, werden wir zunächst ein einfaches Beispiel[<sup>2</sup>](#fn2) durchrechnen, bevor wir das Verfahren auf unseren Datensatz anwenden. Dazu werden wir das <b>Multinomial Naive Bayes</b> Verfahren nutzen, eine der vielen Varianten von Naive Bayes. Der Grund, wieso wir dieses Verfahren nutzen, wird gleich noch erkläutert werden. Schauen wir uns zunächst den Datensatz an. 

<hr style="border: 0.1px solid black;"/>
<span id="fn2" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">2</sup> &nbsp; Dieses Beispiel wurde von dem Naive Bayes Tutorial der Plattform <b>MonkeyLearn</b> übernommen, übersetzt und leicht abgewandelt: https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/.</span>

In [19]:
import pandas as pd

news = {1 : ["Ein großartiges Spiel", "Sport"], 
        2 : ["Das Urteil ist beschlossen", "Kein Sport"],
        3 : ["Ein sehr faires Match", "Sport"],
        4 : ["Ein faires aber langweiliges Spiel", "Sport"],
        5 : ["Es war ein knappes Urteil", "Kein Sport"],
        6 : ["Ein sehr knappes Spiel", "?"]}
simple_df = pd.DataFrame.from_dict(news, orient="index", columns=["Text", "Kategorie"])

simple_df

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es war ein knappes Urteil,Kein Sport
6,Ein sehr knappes Spiel,?


Wir sehen eine Tabelle, in der kurze Nachrichtenartikel in eine von zwei Kategorien eingeteilt wurden: "Sport" oder "Kein Sport". Der sechste Artikel "Ein sehr knappes Spiel" wurde in keine Kategorie eingeteilt. Unsere Aufgabe ist es jetzt, mithilfe des <b>Multinomial Naive Bayes</b> Klassifikationsverfahren die Wahrscheinlichkeiten zu berechnen, ob der Text "Ein sehr knappes Spiel" in die Kategorie "Sport" oder "Kein Sport" eingeteilt wird. Die <u>größere</u> Wahrscheinlichkeit bestimmt, zu welcher Kategorie der Satz gehört.
Mathematisch wird das folgenderweise ausgedrückt: <br>

$ P(Sport|Ein\ sehr\ knappes\ Spiel) $ <br>
$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) $

Wir haben nun ein Problem: Da unsere Daten aus Texten bestehen, können wir nicht mit ihnen rechnen und sie deshalb nicht in die Formel des Bayesschen Theorems einsetzen. Deshalb müssen wir zunächst <b>"Feature Engineering"</b> betreiben, einen zentralen Schritt bei der Nutzung von Machine Learning Algorithmen. 

<div class="alert alert-info">
<b>Exkurs:</b> Feature Engineering

TO-DO: 
</div>

Wir müssen uns dazu entschieden, welche Features wir aus den Texten extrahieren wollen. Würde unsere Datensatz aus Personen bestehen und wir würden eine Klassifikation mit den Klassen "Gesund" und "Nicht Gesund" durchführen wollen, könnten wir als Features die Körpergröße, das Gewicht oder das Geschlecht nehmen und solche Features wie Vornamen, Lieblingsfarbe oder Sprachkenntnisse könnten wir weglassen.<br>
In unserem Fall haben wir jedoch keine numerischen Features, sondern nur Text. Um den Text in die Formel des <b>Bayesschen Theorems</b> einsetzen zu können, müssen wir den Text in Zahlen transformieren. Eine Möglichkeit bestünde darin, die einfachen Worthäufigkeiten jedes Textes zu nehmen. Dieses Verfahren nennt sich <b>Bag-of-Words</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Bag-of-Words Modell

Ein <b>Bag-of-Words</b> Modell (kurz: BoW) ist eine Methode, um Features aus einem Text zu extrahieren, damit man diese für eine Modellierung, wie hier für einen Machine Learning-Algorithmus, verwenden kann. Die Idee hierbei ist, dass Dokumente, die eine ähnliche Anzahl der gleichen Wörter haben, ähnlichen Inhalt haben.

Der Bag-of-Words-Ansatz ist simpel und flexibel und kann vielseitig verwendet werden. Er beschreibt das Auftreten von Wörtern in einem Dokument und besteht aus einem Vokabular der bekannten Wörter und einem Maß für das Vorkommen der bekannten Wörter. Die Bezeichnung "bag" (deutsch: <i>Sack</i>) soll darauf hinweisen, dass alle Informationen über die Struktur oder Reihenfolge der Wörter im Dokument verworfen werden. Es ist also nicht mehr möglich, die Lage eines Wortes nach der Umwandlung eines Textes in einen bag-of-words zu bestimmen.

Das Bag-of-Words-Modell kann erweitert werden, etwa durch <b>tf-idf</b> oder <b>N-Gramme</b> (n-grams).
</div>

Bevor wir aber unseren Text in ein <b>Bag-of-words</b>-Modell überführen, setzen wir zunächst den Satz "Ein sehr knappes Spiel" in die Formel für das <b>Bayessche Theorem</b> ein. Wir erhalten somit:

$ P(Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Sport|Ein\ sehr\ knappes\ Spiel) \ \cdot \ P(Sport)}{P(Sport|Ein\ sehr\ knappes\ Spiel)}  $ <br>


$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) \ \cdot \ P(Kein\ Sport)}{P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel)}  $ <br>


ICH: hier weiter mit dem Divisor problem

liegt an unseren Daten (die den Daten unseres Korpus(?) ähneln). Unsere Daten sind nämlich <b>diskrete</b> Daten. 




Diskrete vs continous... 

Bag of words hat einige Probleme:
- goodnote
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/

Probleme/Erläuterungen für Bericht:
- wie weit Feature Engineering beschreiben, wenn Fokus später auf DL liegt?
- Wieviele Möglichkeiten von BoW beschreiben?
- Beispiel von Monkey Learning war gut, musste aber übersetzt werden. hier ist die deutsche sprache wieder problematisch, wegen den konjugation etc. einige wörter wurden ausgetauscht
- die idee bei diesem teil war es, dass sehr einfach ein paar grundzüge und herangehensweisen von klassifikation erklärt werden können
    - wahl des verfahrens: wurde hier abgenommen, wird später dann genauer erläutert
    - inhalt: was ist das verfahren überhaupt? eine auseinandersetzung mit verfahren ist vorteilhaft, es wurde aber wert darauf gelegt, dass es nicht zu tief in die materie geht, vor allem was den mathematischen aspekt angeht. deshalb wurde die mathematik einfach gehalten und ausreichend erklärt.
    - mathematik: jedes verfahren basiert natürlich auf einer mathematik. geisteswissenschaftler, also auch dhler, scheuen sich aber etwas davor, da der umgang mit formeln nicht geübt ist wie bei informatikern, mathematiker oder allgemein naturwissenschaftlern. hier ist es also wichtig, die mathematik gut und einfach zu erklären und vor allem mathematik, die nicht gebraucht wird, wegzulassen. zu viel sollte m.E. jedoch nicht weggelassen werden, da ein basis verständnis hilft, die verfahren zu verstehen. da in späteren kapiteln der anteil der mathematik zunehmen würde, wurden hier basis rechnung ausführlich erklärt, damit der leser ein verständnis dafür bekommt, ohne abgeschreckt zu werden.
    - info boxen: für diese wurde sich bewusst entschieden, da sie nicht direkt in den text gepasst haben, aber wichtig sind. feature engineering und bow wurden mit einem satz im "fließtext" erklärt und in der info box etwas ausführlicher beleuchtet.
    - einfaches bespiel ohne programmierung: ich finde, dass rechenbeispiele sehr gut helfen, ein verfahren zu verstehen. hätte ich hier direkt mit einem sci kit learn beispiel angefangen, wäre ein tieferes verständnis von naive bayes nicht gegeben. der leser könnte zwar daten klassifizieren, was sci kit learn aber eigentlich im hintergrund macht, bleibt ihm verwehrt. deswegen wurde hier sehr sehr ausfürlich das verfahren per Hand gerechnet. der leser versteht so, was eigentlich gemacht werden und kann sich vorstellen, wieso ein programmiermehtode sinnvoll ist, vor allem bei vielen Daten. dieses einfache beispiel ist sinnbildlich für die funktionsweise von textklassifikationsverfahren. es war jedoch klar/ es wurde sich bewusst dazu entschieden, nicht für jedes weitere verfahren ein händisches rechenbeispiel zu liefern, da dies zu viel des guten wäre und sich zu vieles wiederholen würde. in nächsten verfahren, logistic regression, soll dies noch einmal wiederholt, damit sollte eine gute intuition gegeben sein, was der computer da eigentlich rechnet.
    - sci kit learn:
    - datensatz mit naive bayes:
    - erweiterung mit tf idf und n-gramme: ICH: vllt erst nächstes kapitel?

Quellen:
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/
- https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/